In [ ]:
import csv
import os
from openai import OpenAI

In [ ]:
neo4j_uri = os.environ.get('NEO4J_URI')
neo4j_username = os.environ.get('NEO4J_USERNAME')
neo4j_password = os.environ.get('NEO4J_PASSWORD')
neo4j_auth =  (neo4j_username, neo4j_password)

In [ ]:
open_ai_api_key = os.environ.get('OPENAI_API_KEY')

In [ ]:
client = OpenAI(api_key=open_ai_api_key)

#### Load questions and queries, create 10 questions for each query


In [ ]:
def load_csv_and_generate_questions(input_csv):
    generated_questions = []

    with open(input_csv, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            original_question = row['Input']
            cypher_query = row['Output']
            additional_questions = generate_more_questions(original_question, cypher_query)
            generated_questions.extend(additional_questions)

    # Save the generated questions to a new CSV
    save_to_csv('generated_questions.csv', generated_questions, ['Generated Question'])
    
def load_csv_and_generate_questions(input_csv):
    generated_questions = []

    with open(input_csv, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            original_question = row['Input']
            cypher_query = row['Output']
            additional_questions = generate_more_questions(original_question, cypher_query)
            generated_questions.extend(additional_questions)

    # Save the generated questions to a new CSV
    save_to_csv('generated_questions.csv', generated_questions, ['Generated Question'])

def generate_more_questions(original_question, cypher_query):
    """
    Generate 10 more questions for the given original question that would lead to the same Cypher query.
    """
    prompt = f"Given the Cypher query: {cypher_query}. Generate 10 different natural language questions that would result in this Cypher query. The original question is: {original_question}."
    
    try:
        response = client.chat.completions.create(
            engine="gpt-4",
            prompt=prompt,
            temperature=0.7,
            max_tokens=150,
            n=1,  # Generate 1 set of 10 questions as a single response
            stop="\n"
        )
        questions_text = response.choices[0].text.strip()
        # Split the generated text into individual questions
        additional_questions = [question.strip() for question in questions_text.split('\n') if question.strip()]
        return additional_questions
    except Exception as e:
        print(f"Error generating questions for query: {cypher_query} - {e}")
        return []

def save_to_csv(filename, data, headers):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for item in data:
            writer.writerow([item])

# Replace 'input_questions_queries.csv' with the path to your CSV file
load_csv_and_generate_questions('input_questions_queries.csv')